In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pickle

In [2]:
df = pd.read_csv("D:\Study\Projects\Coffee Roast\coffee_data.csv")

In [3]:
df.head()

,temperature,time,target
0,185.317638,12.693965,1.0
1,259.920475,11.867664,0.0
2,231.013571,14.414242,0.0
3,175.366645,11.720587,0.0
4,187.120865,14.129732,1.0


In [4]:
X = df.loc[:,df.columns != "target"]
y = df.target

print(X.shape, y.shape)

(200, 2) (200,)


In [5]:
norm_layer = tf.keras.layers.Normalization(axis=-1)
norm_layer.adapt(X)

In [6]:
Xt = np.tile(X,(1000,1))
yt= np.tile(y,(1000))   
print(Xt.shape, yt.shape)

(200000, 2) (200000,)


In [7]:
input_size = X.shape[1]

model = Sequential(
    [
        norm_layer,
        Dense(3, activation='sigmoid', input_shape=(input_size,)),
        Dense(1, activation='sigmoid')
    ]
)

model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
)

print(model.summary())

model.fit(Xt, yt, epochs = 10, verbose = 0)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 2)                5         
 n)                                                              
                                                                 
 dense (Dense)               (None, 3)                 9         
                                                                 
 dense_1 (Dense)             (None, 1)                 4         
                                                                 
Total params: 18
Trainable params: 13
Non-trainable params: 5
_________________________________________________________________
None


In [7]:
# inputs = tf.keras.Input(shape=(2,))
# xn = norm_l(inputs)
# outputs = Sequential(
#     [
#         Dense(3, activation='sigmoid', name = 'layer1'),
#         Dense(1, activation='sigmoid', name = 'layer2')
#     ])(xn)
# inference_model = tf.keras.Model(inputs, outputs)
# inference_model.compile(
#     loss = tf.keras.losses.BinaryCrossentropy(),
#     optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
# )
# inference_model.fit(
#     Xt,Yt,            
#     epochs=10
# )

Epoch 1/10
6250/6250 [==============================] - 6s 848us/step - loss: 0.1923
Epoch 2/10
6250/6250 [==============================] - 5s 805us/step - loss: 0.1375
Epoch 3/10
6250/6250 [==============================] - 5s 800us/step - loss: 0.1344
Epoch 4/10
6250/6250 [==============================] - 5s 829us/step - loss: 0.1329
Epoch 5/10
6250/6250 [==============================] - 5s 809us/step - loss: 0.1322
Epoch 6/10
6250/6250 [==============================] - 5s 812us/step - loss: 0.1314
Epoch 7/10
6250/6250 [==============================] - 5s 793us/step - loss: 0.1308
Epoch 8/10
6250/6250 [==============================] - 5s 821us/step - loss: 0.1305
Epoch 9/10
6250/6250 [==============================] - 5s 818us/step - loss: 0.1302
Epoch 10/10
6250/6250 [==============================] - 5s 810us/step - loss: 0.1298


In [8]:
X_input = np.array([200,13.9]).reshape(1,-1)
prediction = model.predict(X_input)
yhat = (prediction >= 0.5).astype(int)
print(prediction)
print(yhat)

1/1 [==============================] - 0s 421ms/step
[[0.9878247]]
[[1]]


In [9]:
# pickle.dump(model, open("coffee_model.pkl", "wb"))

INFO:tensorflow:Assets written to: ram://12f40382-aae9-41b5-855f-fde92f9ee8e0/assets


In [11]:
model.save("./saved_model")

INFO:tensorflow:Assets written to: ./saved_model\assets


In [10]:
# coffee_model = pickle.load(open("coffee_model.pkl", "rb"))

FileNotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for ram://47f938ce-5ba5-41b8-9c3d-c86ad0070195/variables/variables
 You may be trying to load on a different device from the computational device. Consider setting the `experimental_io_device` option in `tf.saved_model.LoadOptions` to the io_device such as '/job:localhost'.